In [2]:
#!pip3 install -r requirements.txt
!python --version

Python 3.7.3


In [ ]:
karpathy_playlist = "https://www.youtube.com/playlist?list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ"

karpathy_urls = get_urls_from_youtube_playlist(karpathy_playlist)
karpathy_playlist_id = extract.playlist_id(karpathy_playlist)

karpathy_playlist_vid_ids = [extract.video_id(url) for url in karpathy_urls]

karpathy_playlist_vid_ids

['VMj-3S1tku0', 'PaCmpygFfXo', 'TCH_1BHY58I', 'P6sfmUTpUmc', 'q8SA3rM6ckI']

In [3]:
# !pip install -r requirements.txt

  Cloning https://github.com/openai/whisper.git to /private/var/folders/tc/q_js7srx33sc183v2vl5lfr00000gn/T/pip-req-build-vzt8jss8
  Running command git clone -q https://github.com/openai/whisper.git /private/var/folders/tc/q_js7srx33sc183v2vl5lfr00000gn/T/pip-req-build-vzt8jss8
  Resolved https://github.com/openai/whisper.git to commit 9f70a352f9f8630ab3aa0d06af5cb9532bd8c21d
     |████████████████████████████████| 137.5 MB 2.7 MB/s eta 0:00:011MB/s eta 0:00:0726.3 MB/s eta 0:00:04| 53.9 MB 26.3 MB/s eta 0:00:042 MB 26.3 MB/s eta 0:00:03��████▊             | 80.7 MB 26.3 MB/s eta 0:00:03eta 0:00:02███████████████████████▏      | 108.1 MB 44.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 112.2 MB/s eta 0:00:01
  Using cached farm_haystack-1.10.0-py3-none-any.whl (736 kB)
  Using cached transformers-4.23.1-py3-none-any.whl (5.3 MB)
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  Using c

In [6]:
import whisper
import datetime
import subprocess
import torch
import numpy as np
import pandas as pd

from haystack.nodes import EmbeddingRetriever, FARMReader
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.pipelines import ExtractiveQAPipeline


In [50]:
transcripts=""
df = pd.read_json(transcripts)
ds = df.to_dict("records")

# document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")
# document_store = ElasticsearchDocumentStore(similarity="cosine", embedding_dim=384)


dicts = [{"content": d["text"], "meta": {i:d[i] for i in d if i!='text'}} for d in ds]

In [60]:
# In-Memory Document Store
from haystack.document_stores import InMemoryDocumentStore, FAISSDocumentStore

# document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", embedding_dim=384, similarity="cosine")

document_store = InMemoryDocumentStore(embedding_dim=384, similarity="cosine")

ValueError: The number of documents present in the SQL database (1) does not match the number of embeddings in FAISS (0). Make sure your FAISS configuration file correctly points to the same database that was used when creating the original index.

In [58]:
document_store.write_documents(dicts)

retriever = EmbeddingRetriever(document_store=document_store, embedding_model='sentence-transformers/all-MiniLM-L6-v2', use_gpu=True, top_k=10)

document_store.update_embeddings(retriever)



reader = FARMReader(model_name_or_path='deepset/roberta-base-squad2', use_gpu=True)

Writing Documents:   0%|          | 0/155 [00:00<?, ?it/s]ERROR:haystack.document_stores.sql:Document c2ae3bae6d60ff5442a064aa2c9bfbc3 - Discarded metadata 'segments', since it has invalid type: dict.
SQLDocumentStore can accept and cast to string only the following types: str, int, float, bool, bytes, bytearray, NoneType
ERROR:haystack.document_stores.sql:Document c2ae3bae6d60ff5442a064aa2c9bfbc3 - Discarded metadata 'segments', since it has invalid type: dict.
SQLDocumentStore can accept and cast to string only the following types: str, int, float, bool, bytes, bytearray, NoneType
ERROR:haystack.document_stores.sql:Document c2ae3bae6d60ff5442a064aa2c9bfbc3 - Discarded metadata 'segments', since it has invalid type: dict.
SQLDocumentStore can accept and cast to string only the following types: str, int, float, bool, bytes, bytearray, NoneType
ERROR:haystack.document_stores.sql:Document c2ae3bae6d60ff5442a064aa2c9bfbc3 - Discarded metadata 'segments', since it has invalid type: dict.
S

RuntimeError: Embedding dimensions of the model (384) don't match the embedding dimensions of the document store (768). Initiate FAISSDocumentStore again with arg embedding_dim=384.

In [ ]:
pipe = ExtractiveQAPipeline(reader, retriever)
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers. 
prediction = pipe.run(
    query="What is self-supervised learning", params={"Retriever": {"top_k": 100}, "Reader": {"top_k": 15}}
)


Inferencing Samples: 100%|██████████| 1/1 [00:10<00:00, 10.81s/ Batches]


In [54]:
prediction

{'query': 'What is self-supervised learning',
 'no_ans_gap': 8.132203578948975,
 'answers': [<Answer {'answer': 'a setting', 'type': 'extractive', 'score': 0.834409236907959, 'context': "0 bits of information per sample. But self-supervised learning here is a setting. You, ideally, we don't know how to do this yet, but ideally you woul", 'offsets_in_document': [{'start': 5337, 'end': 5346}], 'offsets_in_context': [{'start': 71, 'end': 80}], 'document_id': 'c2ae3bae6d60ff5442a064aa2c9bfbc3', 'meta': {'segments': {'id': 0, 'seek': 0, 'start': 0.0, 'end': 8.44, 'text': ' What is self-supervised learning and why is it the dark matter of intelligence?', 'tokens': [50364, 708, 307, 2698, 12, 48172, 24420, 2539, 293, 983, 307, 309, 264, 2877, 1871, 295, 7599, 30, 50786, 50786, 286, 603, 722, 538, 264, 2877, 1871, 644, 13, 50970, 50970, 821, 307, 2745, 257, 733, 295, 2539, 300, 6255, 293, 4882, 366, 884, 300, 321, 4362, 51340, 51340, 366, 406, 11408, 2175, 6108, 365, 8379, 420, 365, 7318, 13, 

In [ ]:
#TODO find timestamp for answer (approximate through start tokens)